In [428]:
from random import randint
from collections import defaultdict

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit

    def __str__(self):
        return f"Value:\t{self.value}\tSuit:\t{self.suit}\n"

    def __repr__(self):
        return f"Value:\t{self.value}\tSuit:\t{self.suit}\n"



class Deck:
    suits = ['s', 'c', 'd', 'h']
    def __init__(self,empty=True):
        self.deck = []

        if empty == False:
            self.populate()

    def __str__(self):
        s = "Your deck:\n"
        for card in self.deck:
            s += str(card)
        return s#"\n" + str(self.deck)

    def insert_card(self, card):
        self.deck.append(card)

    def populate(self):
        for i in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
            for suit in Deck.suits:
                self.deck.append(Card(i, suit))

    def deal(self):
        if len(self.deck) == 0:
            self.populate()

        pop_index = randint(0, len(self.deck)-1)
        card = self.deck.pop(pop_index)

        return card

    def get_value(self):
        return sum([card.value for card in self.deck])

class PokerHand:
    num_board = 5
    num_hole = 2
    def __init__(self):
        self.board = []
        self.hole = []
        deck = Deck(empty=False) # need a Deck to deal from

        for card in range(PokerHand.num_board):
            self.board.append(deck.deal())
        for card in range(PokerHand.num_hole):
            self.hole.append(deck.deal())

        #hole_evaluate()
        #board_evaluate
        #full_evaluate - i suspect this wont be necessary
        return

    def hole_evaluate(self):
        # high
        self.hole_high = max(self.hole.value[0], self.hole.value[1])

        # pairs
        self.hole_isPair = self.hole.value[0] == self.hole.value[1]

        # suits
        self.hole_isSuited = self.hole.suit[0] == self.hole.suit[1]

        # suitedConnector (e.g Jack D & Queen D)
        self.hole_isSuitedConnector = False
        if isSuited:
            if abs(self.hole.value[0] - self.hole.value[1]) == 1:
                isSuitedConnector = True


    def evaluate(self, cards: [Card]):
        d = build_face_counts(cards)

        straight_flush = isStraightFlush(cards)
        if straight_flush:
            print(f"Straight Flush: {straight_flush} high")
            return

        quads = isQuads(d)
        if quads:
            print(f"Quads: 4x{quads}")
            return

        fh = isFullHouse(d)
        if fh[0]:
            print(f"FullHouse: 3x{fh[0]} and 2x{fh[1]}")
            return

        flush = isFlush(cards)
        if flush:
            print(f"Flush: {flush}")
            return

        straight = isStraight(cards)
        if straight:
            print(f"Straight: {straight} high")
            return

        trips = isTrips(d)
        if trips:
            print(f"Trips: 3x{trips}")
            return

        tpmax, tpmin = isTwoPair(d)
        if  tpmax:
            print(f"Two Pair: 2x{tpmax} and 2x{tpmin}")
            return
        pair = isPair(d)
        if pair:
            print(f"One Pair: 2x{pair}")
            return

# Basic Hand Functions

def build_face_counts(deck: [Card]) -> dict:
    d = defaultdict(int)
    for card in deck:
        d[card.value] += 1
    return d

def isTrips(counts: dict) -> int:
    for value in sorted(counts.keys(), reverse=True):
        if counts[value] == 3:
            return value
    return 0

def isPair(counts: dict) -> int:
    for value in sorted(counts.keys(), reverse=True):
        if counts[value] == 2:
            return value
    return 0

def isQuads(counts: dict) -> int:
    for value in sorted(counts.keys(), reverse=True):
        if counts[value] == 4:
            return value
    return 0

def isTwoPair(counts: dict) -> (int, int):
    dubs = []
    for value in sorted(counts.keys(), reverse=True):
        if counts[value] == 2 and value not in dubs: # Prevent duplicate dubs
            dubs.append(value)
    if len(dubs) >= 2 and dubs[0] != dubs[1]: # Quads != TwoPair
        return max(dubs[0], dubs[1]), min(dubs[0], dubs[1])
    return 0, 0

def isFullHouse(counts: dict) -> (int, int):
    trips = 0
    dubs = 0
    print(sorted(counts.keys(), reverse=True))
    for value in sorted(counts.keys(), reverse=True):
        if counts[value] == 3: # Get biggest trip, no duplicates
            trips = value
        if counts[value] >= 2: # Get biggest dub, no duplicates
            dubs = value
        if trips != 0 and dubs != 0:
            return trips, dubs
    return 0,0

# Advanced Hand Functions

def isFlush(deck: [Card]) -> int: # (hole, board)
    polynomial = 0
    # Dictionary stores number of cards of a given suit
    suit_count = {'d': 0, 's': 0, 'c': 0, 'h':0}
    for card in deck:
        suit_count[card.suit] += 1

    for suit in Deck.suits:
        if suit_count[suit]>= 5:
            polynomial = build_poly(deck, suit)

    return polynomial

def build_poly(cards: [Card], suit: str) -> int:
    cards.sort(key=lambda x: x.value, reverse=True)
    sum = 0
    pow = 4
    for card in cards:
        if card.suit != suit or pow < 0:
            print(f"Disregarding: {card.value} {card.suit}")
            continue
        sum += (card.value * (13 ** pow))
        pow -= 1
    return sum

def isStraight(deck: [Card]) -> int:
    deck.sort(key=lambda x: x.value, reverse=True)
    unique_values = sorted(set([card.value for card in deck]), reverse=True)

    if 14 in unique_values:
        unique_values.append(1)

    if len(unique_values) < 5:
        return 0

    i = 0
    while (i + 4 < len(unique_values)) and (unique_values[i] - unique_values[i+4] != 4):
        i+=1

    if (i + 4 < len(unique_values)) and unique_values[i] - unique_values[i+4] == 4:
        return unique_values[i]

    return 0

def isStraightFlush(deck: [Card]) -> int:
    l = defaultdict(list)
    for suit in Deck.suits:
        l[suit] = []

    for card in deck:
        l[card.suit].append(card)

    return max([isStraight(l[suit]) for suit in Deck.suits])


def debug_test():
    p = PokerHand()
    cards = [Card(2, 'd'), Card(3, 'd'), Card(4, 'd'), Card(5, 'd'), Card(14, 'd')]
    print(cards)
    p.evaluate(cards)

debug_test()

def auto_test():
    p = PokerHand()
    d = Deck(empty=False)
    cards = [d.deal() for i in range(7)]
    print(cards)
    p.evaluate(cards)


[Value:	2	Suit:	d
, Value:	3	Suit:	d
, Value:	4	Suit:	d
, Value:	5	Suit:	d
, Value:	14	Suit:	d
]
Straight Flush: 5 high
